<a href="https://colab.research.google.com/github/tacerihsehc613/ML-board/blob/main/naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#stuff to import
import pandas as pd
import numpy as np
import random
import sklearn
from sklearn.model_selection import train_test_split
!pip install pandas==1.2.1
!pip install numpy==1.19.4
!pip install scikit-learn==0.24.0

In [ ]:
assert pd.__version__ == "1.2.1", "Looks like you don't have the same version of pandas as us!"
assert np.__version__ == "1.19.4", "Looks like you don't have the same version of numpy as us!"
assert sklearn.__version__ == "0.24.0", "Looks like you don't have the same version of sklearn as us!"


In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/Colab Notebooks
os.listdir('.')

Mounted at /content/drive/
/content/drive/MyDrive/Colab Notebooks


['Tweets-BarackObama.csv',
 'trump_20200530.csv',
 'train.csv',
 'sample_solution.csv',
 'test.csv',
 'Practical1_Assignment.ipynb의 사본',
 'Practical2_Assignment.ipynb',
 'src',
 'KNN.ipynb',
 'Project1.ipynb',
 'naive_bayes.ipynb']

In [ ]:
df_t = pd.read_csv('trump_20200530.csv')
trump_tweets = df_t['text']
df_t = pd.read_csv('Tweets-BarackObama.csv')
obama_tweets = df_t['Tweet-text']

tweet_data = trump_tweets.append(obama_tweets, ignore_index=True)
tweet_labels = np.array(['T' for _ in range(len(trump_tweets))] + ['O' for _ in range(len(obama_tweets))])

In [ ]:
lab, counts = np.unique(tweet_labels, return_counts=True)
print('Number of tweets about ', lab[0], ': ', counts[0])
print('Number of tweets about ', lab[1], ': ', counts[1])

Number of tweets about  O :  6851
Number of tweets about  T :  18467


In [ ]:
from typing import Type
from sklearn.model_selection import train_test_split
#Split data into train_tweets, test_tweets, train_labels and test_labels  
train_tweets, test_tweets, train_labels, test_labels = train_test_split(tweet_data,tweet_labels)
print(train_tweets)
print(type(train_tweets))
print(train_tweets[0])
print(train_labels)
print(type(train_labels))
print(train_labels[0])

6816     RT @GOPoversight: • No quid pro quo• No pressu...
13013    We are pleased to announce that Matthew G. Whi...
3211     RT @SenateGOP: American workers are WINNING an...
7808     The Whistleblower’s complaint is completely di...
24412    “We may be momentarily knocked off our feet bu...
                               ...                        
3055     RT @DanScavino: “I’m a BIG FAN of @realDonaldT...
8228     .@CNN &amp; @MSNBC were all set to have a BIG ...
17270    I will be holding a major briefing on the Opio...
6128                               https://t.co/ru2n7i2gzu
24349    While many in Congress continue to deny the sc...
Length: 18988, dtype: object
<class 'pandas.core.series.Series'>
RT @ScottAdamsSays: Malaria drug and zinc the missing link https://t.co/FC9CpuH0Yr via @theconwom
['T' 'T' 'T' ... 'T' 'T' 'O']
<class 'numpy.ndarray'>
T


In [ ]:
P_O=len(tweet_labels[tweet_labels == 'O']) / len(tweet_labels)
P_T=len(tweet_labels[tweet_labels == 'T']) / len(tweet_labels)
print(P_O,P_T)

0.2705979935223951 0.7294020064776049


In [ ]:
word_counter = dict()
for (tweet, label) in zip(train_tweets, train_labels):
  tweet_w = tweet.strip().split(' ')
  tweet_w=list(set(tweet_w))           
  for i in range(len(tweet_w)):
    if tweet_w[i].lower() not in word_counter:
      word_counter[tweet_w[i].lower()]={"O":0, "T":0}
      word_counter[tweet_w[i].lower()][label]+=1
    else:
      word_counter[tweet_w[i].lower()][label]+=1
print(len(word_counter))
print(word_counter['rt'])

44164
{'O': 52, 'T': 4793}


In [ ]:
nr_of_words_to_use = 100
popular_words = sorted(word_counter.items(), key=lambda x: x[1]['O'] + x[1]['T'], reverse=True)
popular_words = [x[0] for x in popular_words[:nr_of_words_to_use]]
print(popular_words)

['the', 'to', 'and', 'of', 'a', 'in', 'rt', 'is', 'for', 'on', 'that', 'our', 'are', 'with', 'we', 'will', 'be', 'i', 'this', 'president', 'great', 'you', 'have', 'it', 'obama', 'at', 'they', '&amp;', 'has', 'was', 'all', 'not', 'by', 'from', 'he', 'people', '—president', 'just', 'who', 'as', 'my', 'very', 'your', 'more', 'about', 'so', 'no', 'their', 'if', 'thank', 'democrats', 'do', 'but', 'get', 'now', 'an', 'his', 'new', 'than', 'trump', 'what', 'up', '-', 'out', 'been', 'time', 'should', '', 'big', 'american', 'news', 'or', 'make', 'can', 'fake', 'many', 'would', 'never', 'today', 'one', 'country', '@realdonaldtrump', 'there', 'want', 'when', 'u.s.', 'america', 'house', '@realdonaldtrump:', 'good', 'congress', 'going', 'united', 'how', 'like', 'even', 'much', 'me', 'states', 'only']


In [ ]:
P_w_given_t = {}
P_w_given_o = {}

for word in popular_words:
  P_w_given_t[word] = word_counter[word]['T'] / len(tweet_labels[tweet_labels == 'T'])
  P_w_given_o[word] = word_counter[word]['O'] / len(tweet_labels[tweet_labels == 'O'])


In [ ]:
classifier = {
    'basis'  : popular_words,
    'P(T)'   : P_O,
    'P(O)'   : P_T,
    'P(w|O)' : P_w_given_o,
    'P(w|T)' : P_w_given_t
    }  

In [ ]:
def tweet_classifier(tweet, classifier_dict):
  Trump=1.0
  Obama=1.0

  words_in_tweet = np.unique([x.lower() for x in tweet.split()])
  for i in classifier_dict['basis']:
    if(i in words_in_tweet):
      #print(classifier_dict['P(w|T)'][i])
      Trump = Trump * classifier_dict['P(w|T)'][i]
      Obama = Obama * classifier_dict['P(w|O)'][i]
    else:
      Trump *= (1-classifier_dict['P(w|T)'][i])
      Obama *= (1-classifier_dict['P(w|O)'][i])
  Trump *= classifier_dict['P(T)']
  Obama *= classifier_dict['P(O)']
  
  if(Trump > Obama): return 'T'
  else: return 'O'
    
# ... Code for classifying tweets using the naive bayes classifier

In [ ]:
def test_classifier(classifier, test_tweets, test_labels):
    total = len(test_tweets)
    correct = 0
    for (tweet,label) in zip(test_tweets, test_labels):
        predicted = tweet_classifier(tweet,classifier)
        if predicted == label:
            correct = correct + 1
    return(correct/total)
  

In [ ]:
acc = test_classifier(classifier, test_tweets, test_labels)
print(f"Accuracy: {acc:.4f}")

Accuracy: 0.8521
